In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import os
import random

## Step 1 - Dataset Augmentation for SimCLR

In [2]:
class SimCLRDataTransform:
    """
    Yeh transform class ek image leti hai aur uske do augmented versions (views) return karti hai.
    """
    def __init__(self, image_size=128):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(size=image_size, scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            # Color jitter SimCLR ke liye bahut zaroori hai
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1),
            transforms.RandomGrayscale(p=0.2),
            transforms.GaussianBlur(kernel_size=23, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __call__(self, image):
        # Ek hi image par do baar alag-alag transform apply karein
        view_1 = self.transform(image)
        view_2 = self.transform(image)
        return view_1, view_2

class SatelliteImageDataset(Dataset):
    """
    Loads the FULL satellite images.
    The SimCLR transform will handle the random cropping (tiling).
    """
    def __init__(self, image_dir, transform):
        self.image_dir = image_dir
        # Just get a list of all image paths
        self.image_files = []
        for f in os.listdir(image_dir):
            if f.endswith(('.png', '.jpg', '.jpeg')):
                self.image_files.append(os.path.join(image_dir, f))
                
        self.transform = transform
        
        print(f"Dataset created with {len(self.image_files)} images.")

    def __len__(self):
        # The length is just the number of images
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        
        try:
            # Open the full image
            image = Image.open(image_path).convert("RGB")
            
            # Apply the SimCLR transform (which returns view_1, view_2)
            # The transform's RandomResizedCrop will create the "tile"
            return self.transform(image)
            
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # Fallback: get a different random image
            random_idx = random.randint(0, len(self)-1)
            return self.__getitem__(random_idx)

In [3]:
IMAGE_SIZE = 256
DATA_DIR = "/kaggle/input/solar-dataset/Dataset/"
BATCH_SIZE = 64

In [4]:
# 1. Create the transform
# This transform *is* your tiling logic
transform = SimCLRDataTransform(image_size=IMAGE_SIZE)

# 2. Create the Dataset (with the simplified class)
dataset = SatelliteImageDataset(image_dir=DATA_DIR, transform=transform)

Dataset created with 2516 images.


In [5]:
train_size = int(0.9*len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

# 3. Create the DataLoader
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

print(f"Data split: {len(train_ds)} train, {len(val_ds)} val")

Data split: 2264 train, 252 val


## Step 2 - SimCLR Model aur Training

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50
import torch.optim as optim
from tqdm import tqdm
import numpy as np

In [7]:
class EncoderProjector(nn.Module):
    """
    1. Encoder (Backbone): ResNet-50.
    2. Projection Head: MLP.
    """
    def __init__(self, encoder_output_dim=2048, projection_dim=128):
        super(EncoderProjector, self).__init__()
        
        # 1. Encoder (ResNet-50)
        self.encoder = resnet50(weights='DEFAULT')
        num_features = self.encoder.fc.in_features
        self.encoder.fc = nn.Identity() # Aakhri layer hata di
        
        # 2. Projection Head (g)
        self.projector = nn.Sequential(
            nn.Linear(num_features, num_features),
            nn.ReLU(),
            nn.Linear(num_features, projection_dim)
        )

    def forward(self, x):
        features = self.encoder(x)
        projections = self.projector(features)
        return projections

In [8]:
class MoCo(nn.Module):
    """
    MoCo (Momentum Contrast) implementation.
    Yeh queue aur ek momentum encoder (encoder_k) istemal karta hai.
    """
    def __init__(self, base_encoder_cls, dim=128, K=4096, m=0.999, T=0.07):
        """
        dim: feature dimension (aapka PROJECTION_DIM)
        K: queue size (kitne negative samples store karne hain)
        m: momentum (key encoder ko update karne ke liye)
        T: temperature
        base_encoder_cls: Aapki 'EncoderProjector' class
        """
        super(MoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T

        # Do encoders banayein: query aur key
        self.encoder_q = base_encoder_cls(projection_dim=dim)
        self.encoder_k = base_encoder_cls(projection_dim=dim)

        # Key encoder (encoder_k) ke parameters ko copy karein
        # Aur key encoder ke liye gradient calculation band kar dein
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)
            param_k.requires_grad = False

        # Negative samples ki queue banayein
        # 'register_buffer' istemal hota hai takeh yeh model state ka hissa ho,
        # lekin optimizer isse update na kare
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = F.normalize(self.queue, dim=0)
        
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Key encoder ko slowly update karein (momentum update)
        encoder_k = m * encoder_k + (1-m) * encoder_q
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1.0 - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        # 'keys' current batch ke keys hain (shape: [B, dim])
        batch_size = keys.shape[0]
        ptr = int(self.queue_ptr)
        
        # Queue mein naye keys daalein
        self.queue[:, ptr : ptr + batch_size] = keys.T
        
        # Pointer ko aage badhayein
        ptr = (ptr + batch_size) % self.K
        self.queue_ptr[0] = ptr

    def forward(self, im_q, im_k):
        """
        Input:
        im_q: view 1 ka batch (query)
        im_k: view 2 ka batch (key)
        """
        
        # 1. Query features compute karein (gradients ke saath)
        q = self.encoder_q(im_q) # Shape: [B, dim]
        q = F.normalize(q, dim=1) # Normalize

        # 2. Key features compute karein (gradients nahi chahiye)
        with torch.no_grad():
            self._momentum_update_key_encoder() # Key encoder ko update karein
            
            k = self.encoder_k(im_k) # Shape: [B, dim]
            k = F.normalize(k, dim=1) # Normalize

        # 3. Positive logits calculate karein (query vs positive key)
        l_pos = (q * k).sum(dim=1).unsqueeze(-1) # Shape: [B, 1]

        # 4. Negative logits calculate karein (query vs negative keys in queue)
        l_neg = torch.matmul(q, self.queue.clone().detach()) # Shape: [B, K]

        # 5. Logits ko milayein (Positive sample pehle)
        logits = torch.cat([l_pos, l_neg], dim=1) # Shape: [B, 1+K]

        # 6. Temperature scale karein
        logits /= self.T

        # 7. Labels banayein (pehla sample [index 0] hamesha positive hota hai)
        labels = torch.zeros(logits.shape[0], dtype=torch.long).to(q.device)
        
        # 8. Current batch ke keys ko queue mein daalein
        self._dequeue_and_enqueue(k)

        return logits, labels

In [9]:
NUM_EPOCHS = 100
PATIENCE = 5
PROJECTION_DIM = 128
LEARNING_RATE = 1e-4 # MoCo ke liye learning rate different ho sakti hai
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# MoCo-specific parameters
QUEUE_SIZE = 4096   # Negative samples ka size (Batch size se bada)
MOMENTUM = 0.999    # Momentum encoder ke liye
TEMPERATURE = 0.07  # MoCo mein yeh common hai

In [10]:
# 1. Model initialize karein
model = MoCo(
    base_encoder_cls=EncoderProjector,
    dim=PROJECTION_DIM,
    K=QUEUE_SIZE,
    m=MOMENTUM,
    T=TEMPERATURE
).to(DEVICE)

# 2. Loss function (Siraf CrossEntropy)
criterion = nn.CrossEntropyLoss()

# 3. Optimizer
# Note: Hum sirf 'encoder_q' ke parameters optimize karte hain
# 'encoder_k' momentum se update hota hai
optimizer = optim.Adam(model.encoder_q.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 227MB/s]


In [11]:
epoch_no_improve = 0
best_val_loss = np.inf
best_model_path = "moco_encoder_best.pth"

In [12]:
scaler = torch.cuda.amp.GradScaler()

try:
    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0

        for (views_1, views_2) in tqdm(train_loader, desc=f"Epoch {epoch+1} (Train)"):
            im_q = views_1.to(DEVICE) # Query images
            im_k = views_2.to(DEVICE) # Key images
            
            # Forward pass
            # Model ab 'logits' aur 'labels' return karega
            with torch.amp.autocast(DEVICE):
                logits, labels = model(im_q, im_k)
                # Loss calculate karein
                loss = criterion(logits, labels)
            
            # Backward pass aur optimization
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            
        # --- Ek Epoch poora ho gaya ---
        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for (views_1, views_2) in tqdm(val_loader, desc=f"Epoch {epoch+1} (Val)"):
                im_q = views_1.to(DEVICE)
                im_k = views_2.to(DEVICE)

                with torch.amp.autocast(DEVICE):
                    logits, labels = model(im_q, im_k)
                    loss = criterion(logits, labels)
                
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{NUM_EPOCHS} Complete.")
        print(f"Avg Train Loss: {avg_train_loss:.4f} | Avg Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            print(f"Validation loss improved ({best_val_loss:.4f} --> {avg_val_loss:.4f}). Saving model...")
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Behtareen model ko save karein
            torch.save(model.encoder_q.encoder.state_dict(), best_model_path)
        else:
            epochs_no_improve += 1
            print(f"No improvement in validation loss for {epochs_no_improve} epoch(s).")
        
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break
        
        print("-" * 30)
        
    # --- Saari training poori ho gayi ---
    print("Training finished.")
    print(f"Best model saved to {best_model_path} with validation loss: {best_val_loss:.4f}")
    
except Exception as e:
    print(f"An error occurred during training step: {e}")
    if 'cuda' in str(e).lower():
        print("Hint: Agar CUDA out of memory hai, to BATCH_SIZE kam karein.") 

/tmp/ipykernel_19/3735282835.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1 (Val): 100%|██████████| 3/3 [00:16<00:00,  5.47s/it]


Epoch 1/100 Complete.
Avg Train Loss: 5.1426 | Avg Val Loss: 5.5059
Validation loss improved (inf --> 5.5059). Saving model...
------------------------------


Epoch 2 (Val): 100%|██████████| 3/3 [00:16<00:00,  5.34s/it]


Epoch 2/100 Complete.
Avg Train Loss: 5.9090 | Avg Val Loss: 5.7847
No improvement in validation loss for 1 epoch(s).
------------------------------


Epoch 3 (Val): 100%|██████████| 3/3 [00:16<00:00,  5.37s/it]


...

Epoch 41/100 Complete.
Avg Train Loss: 1.1831 | Avg Val Loss: 1.3941
No improvement in validation loss for 3 epoch(s).
------------------------------


Epoch 42 (Val): 100%|██████████| 3/3 [00:14<00:00,  4.88s/it]


Epoch 42/100 Complete.
Avg Train Loss: 1.1714 | Avg Val Loss: 1.5195
No improvement in validation loss for 4 epoch(s).
------------------------------


Epoch 43 (Val): 100%|██████████| 3/3 [00:14<00:00,  4.91s/it]

Epoch 43/100 Complete.
Avg Train Loss: 1.1315 | Avg Val Loss: 1.4299
No improvement in validation loss for 5 epoch(s).
Early stopping triggered after 43 epochs.
Training finished.
Best model saved to moco_encoder_best.pth with validation loss: 1.3816
